In [1]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time

warnings.filterwarnings("ignore")

In [2]:
paths = os.listdir(path='F:\\')
bb_path = list(filter(lambda file: file.endswith('BB.csv'), paths))[0]
hh_path = list(filter(lambda file: file.endswith('HH.csv'), paths))[0]
print(bb_path, hh_path)

SVO-AER_2018-01-01--2019-12-31BB.csv SVO-AER_2018-01-01--2019-12-31HH.csv


In [3]:
use_cols = ('SDAT_S', 'FLT_NUM', 'DD', 'PASS_BK', 'PASS_DEP', 'NS', 'DTD')
dtypes = {'FLT_NUM': 'float16', 'PASS_BK': 'float16',
        'PASS_DEP': 'float16', 'NS': 'float16', 'DTD': 'float16'}
dates = ['SDAT_S', 'DD']
hh = pd.read_csv('F:\\' + hh_path, delimiter=';', dtype=dtypes, parse_dates=dates, usecols=use_cols)
size_gb = f'{sys.getsizeof(hh) / 1024 / 1024 / 1024} Gb'
print(size_gb)
display(hh.head())

1.388582531362772 Gb


,SDAT_S,FLT_NUM,DD,PASS_BK,PASS_DEP,NS,DTD
0,2018-01-02,1126.0,2018-01-01,12.0,12.0,0.0,-1.0
1,2018-01-02,1126.0,2018-01-01,0.0,0.0,0.0,-1.0
2,2018-01-02,1126.0,2018-01-01,3.0,3.0,0.0,-1.0
3,2018-01-02,1126.0,2018-01-01,0.0,0.0,0.0,-1.0
4,2018-01-02,1126.0,2018-01-01,0.0,0.0,0.0,-1.0


In [4]:
hh.dropna(how='any', axis=0, inplace=True)
hh[['PASS_BK', 'PASS_DEP', 'NS', 'DTD']] = hh[['PASS_BK', 'PASS_DEP', 'NS', 'DTD']].astype(int)
hh.to_csv('F:\\HH__SDAT_S__FLT_NUM__DD__SEG_CLASS_CODE.csv', index=False)

In [3]:
%%time
dtypes = {'FLT_NUM': 'float16'}
dates = ['SDAT_S', 'DD']
hh = pd.read_csv('F:\\HH__SDAT_S__FLT_NUM__DD__SEG_CLASS_CODE.csv', dtype=dtypes, parse_dates=dates)
hh['FLT_NUM'] = hh['FLT_NUM'].astype(int)
display(hh.head())

,SDAT_S,FLT_NUM,DD,PASS_BK,PASS_DEP,NS,DTD
0,2018-01-02,1126,2018-01-01,12,12,0,-1
1,2018-01-02,1126,2018-01-01,0,0,0,-1
2,2018-01-02,1126,2018-01-01,3,3,0,-1
3,2018-01-02,1126,2018-01-01,0,0,0,-1
4,2018-01-02,1126,2018-01-01,0,0,0,-1


Wall time: 1min 30s


In [4]:
print('Все рейсы из списка', sorted(hh['FLT_NUM'].unique()))

Все рейсы из списка [112, 1116, 1118, 1120, 1122, 1124, 1125, 1126, 1128, 1130, 1132, 1134, 1136, 1138, 1140, 1148, 1152, 1740, 1772, 1780, 1782, 1784, 1786, 1788, 1790, 1792, 1794, 1796, 1798, 2980, 2990, 6180, 6184]


In [5]:
use_cols = ('FLTDAT', 'FLTTIME', 'DAIS', 'FLTNUM', 'BCLS', 'TARIF_RUB')
dates = ['FLTDAT', 'DAIS']
dtypes = {'FLTNUM': 'int16', 'TARIF_RUB': 'float16'}
bb = pd.read_csv('F:\\' + bb_path, delimiter=';', dtype=dtypes, parse_dates=dates, usecols=use_cols)
bb = bb.dropna(how='any', axis=0)
bb['FLTTIME'] = pd.to_timedelta(bb['FLTTIME'])
bb['DEPARTURE'] = bb['FLTDAT'] + bb['FLTTIME']
bb.head()

,FLTDAT,FLTTIME,DAIS,FLTNUM,BCLS,TARIF_RUB,DEPARTURE
0,2018-01-21,0 days 13:35:00,2018-01-20,1124,E,2740.0,2018-01-21 13:35:00
1,2018-01-10,0 days 09:10:00,2017-11-13,1122,L,4744.0,2018-01-10 09:10:00
2,2018-01-11,0 days 09:10:00,2017-09-30,1122,N,1530.0,2018-01-11 09:10:00
3,2018-01-26,0 days 17:35:00,2017-09-30,1140,N,1530.0,2018-01-26 17:35:00
4,2018-01-06,0 days 09:10:00,2017-09-12,1122,N,1897.0,2018-01-06 09:10:00


# Сезонный профиль

Исполнитель: Кондрашов Д. М.

In [6]:
%%time
flight = 1126  # input()
date = datetime.strptime('2019-11-27', '%Y-%m-%d')  # input()
flttime = pd.Timedelta('21:20:00')  # input()
print(date.date(), str(flttime).replace('0 days ', ''), date.strftime("%A"))
one_day = timedelta(days=1)
date_minus_52_weeks = date - timedelta(weeks=52)
date_minus_53_weeks = date_minus_52_weeks - timedelta(weeks=1)
date_minus_51_weeks = date_minus_52_weeks + timedelta(weeks=1)
# 52 недели назад +- 1 день
date_minus_52_weeks_gap = [date_minus_52_weeks,
                           date_minus_52_weeks + one_day, date_minus_52_weeks - one_day]
# 53 недели назад +- 1 день
date_minus_53_weeks_gap = [date_minus_53_weeks,
                           date_minus_53_weeks + one_day, date_minus_53_weeks - one_day]
# 51 неделя назад +- 1 день
date_minus_51_weeks_gap = [date_minus_51_weeks,
                           date_minus_51_weeks + one_day, date_minus_51_weeks - one_day]
print(date_minus_52_weeks.date(), date_minus_53_weeks.date(), date_minus_51_weeks.date(), sep='\n')
# фильтрация по дням
filtered_hh = hh[(hh['DD'].isin(date_minus_52_weeks_gap)) |
                 (hh['DD'].isin(date_minus_53_weeks_gap)) |
                 (hh['DD'].isin(date_minus_51_weeks_gap))]
print(f"Отфильтрованных данных: {filtered_hh.shape[0]:_}")
merged = pd.merge(bb, filtered_hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'], how='inner')
merged = merged.dropna(how='any', axis=0)
print(f"Объединённых данных: {merged.shape[0]:_}")
two_hours = pd.Timedelta('2:00:00')
# 52 недели назад в заданное время +- 2 часа
date_minus_52_weeks_gap = [date_minus_52_weeks + flttime + two_hours,
                           date_minus_52_weeks + flttime - two_hours]
# 53 недели назад в заданное время +- 2 часа
date_minus_53_weeks_gap = [date_minus_53_weeks + flttime + two_hours,
                           date_minus_53_weeks + flttime - two_hours]
# 51 неделя назад в заданное время +- 2 часа
date_minus_51_weeks_gap = [date_minus_51_weeks + flttime + two_hours,
                           date_minus_51_weeks + flttime - two_hours]
# фильтрация по времени +- 2 часа
merged = merged[(date_minus_52_weeks_gap[0] >= merged['DEPARTURE']) &
                (date_minus_52_weeks_gap[1] <= merged['DEPARTURE']) |
                (date_minus_53_weeks_gap[0] >= merged['DEPARTURE']) &
                (date_minus_53_weeks_gap[1] <= merged['DEPARTURE']) |
                (date_minus_51_weeks_gap[0] >= merged['DEPARTURE']) &
                (date_minus_51_weeks_gap[1] <= merged['DEPARTURE'])]
merged = merged.drop(columns=['DD', 'FLT_NUM'], axis=1)
merged = merged.dropna(how='any')
print(f"Отфильрованных по времени данных: {merged.shape[0]:_}")

2019-11-27 21:20:00 Wednesday
2018-11-28
2018-11-21
2018-12-05
Отфильтрованных данных: 852_691
Объединённых данных: 31_219_147
Отфильрованных по времени данных: 2_243_490
Wall time: 20.6 s


## Уникальные рейсы (рейс/дата/время) в сезонном профиле:

In [7]:
unique_flights = (merged['FLTNUM'].astype(str) + ' ' +
                 (merged['FLTDAT'] + merged['FLTTIME']).astype(str)).unique()
print(f'{len(unique_flights)} уникальных рейсов: {unique_flights}')

6 уникальных рейсов: ['1128 2018-11-28 21:20:00' '1130 2018-11-21 23:00:00'
 '1128 2018-11-21 21:20:00' '1130 2018-11-28 23:00:00'
 '1128 2018-12-05 21:20:00' '1130 2018-12-05 23:00:00']


Окно в 4 часа даёт 6 уникальных рейсов. Окно в 6 часов - больше 10.

## Средние цены тарифов для классов в сезонном профиле:

In [9]:
bcls_mean = merged[['BCLS', 'TARIF_RUB']].copy()
bcls_mean = bcls_mean.groupby(by=['BCLS'])['TARIF_RUB'].mean().reset_index()
bcls_mean

,BCLS,TARIF_RUB
0,D,29504.0
1,E,4604.0
2,L,7692.0
3,N,2394.0
4,O,5700.0
5,Q,7136.0
6,R,863.5
7,T,5716.0
8,X,0.0


## Среднее количество и std броней (PASS_BK) в разрезе классов (BCLS) и дней до вылета (DTD):

In [27]:
mean_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                              aggfunc=['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_pass_bk s ' + file_name
mean_pass_bk.to_csv(file_name)
mean_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
mean_pass_bk = mean_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(mean_pass_bk)

std_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                             aggfunc=['std'])
file_name = f's {str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'std_pass_bk s ' + file_name
std_pass_bk.to_csv(file_name)
std_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
std_pass_bk = std_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(std_pass_bk)

BCLS,D,E,L,N,O,Q,R,T,X
DTD,,,,,,,,,
-1,4.602273,5.813131,6.227273,4.875931,5.227273,6.227273,4.375668,6.118182,5.500000
0,4.602273,5.813131,6.227273,4.875931,5.227273,6.227273,4.375668,6.118182,5.500000
1,3.840909,5.944444,7.227273,4.131893,3.977273,7.227273,3.024064,6.518182,5.750000
2,3.022727,5.666667,7.136364,3.423994,3.113636,7.136364,2.310160,6.272727,5.272727
3,2.306818,5.434343,6.909091,2.944486,2.545455,6.909091,1.990642,6.018182,4.772727
...,...,...,...,...,...,...,...,...,...
213,0.127273,0.090909,0.090909,0.094995,0.090909,0.090909,0.097403,0.090909,0.090909
214,0.127273,0.090909,0.090909,0.094995,0.090909,0.090909,0.097403,0.090909,0.090909
215,0.127273,0.090909,0.090909,0.094995,0.090909,0.090909,0.097403,0.090909,0.090909


BCLS,D,E,L,N,O,Q,R,T,X
DTD,,,,,,,,,
-1,12.577113,14.705931,15.538821,13.292429,13.948779,15.562526,11.963458,15.368699,14.524652
0,12.559184,14.687304,15.521113,13.291190,13.908869,15.532912,11.961458,15.333571,14.483094
1,11.171684,16.915035,19.501607,12.425733,11.255959,19.516432,8.860716,18.108618,16.345030
2,8.698793,16.653429,19.525803,10.810140,8.745849,19.540646,6.672553,17.922160,15.488043
3,6.391515,16.528402,19.515566,9.838130,6.972450,19.530401,5.615917,17.830753,14.860669
...,...,...,...,...,...,...,...,...,...
213,0.489321,0.417125,0.417072,0.425455,0.418185,0.417389,0.430573,0.417548,0.418185
214,0.489321,0.417125,0.417072,0.425455,0.418185,0.417389,0.430573,0.417548,0.418185
215,0.489321,0.417125,0.417072,0.425455,0.418185,0.417389,0.430573,0.417548,0.418185


## Среднее количество улетевших пассажиров (PASS_DEP) в разрезе классов (BCLS) и дней до вылета (DTD) в момент времени -1:

In [30]:
mean_std_pass_dep = merged[merged['DTD'] == -1]
mean_std_pass_dep = mean_std_pass_dep.groupby(by=['BCLS'])['PASS_DEP'].agg(['mean', 'std'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_std_pass_dep s ' + file_name
mean_std_pass_dep.to_csv(file_name)
mean_std_pass_dep = pd.read_csv(file_name, index_col=0)
display(mean_std_pass_dep)

,mean,std
BCLS,,
D,4.579545,12.539056
E,5.732323,14.479254
L,6.227273,15.538821
N,4.764158,12.999379
O,5.045455,13.392320
Q,6.227273,15.562526
R,4.193850,11.416757
T,6.045455,15.174451
X,5.500000,14.524652


## Среднее количество неявившихся пассажиров (NS) в разрезе классов (BCLS) и дней до вылета (DTD):

In [35]:
mean_ns = merged[merged['DTD'] == -1]
mean_ns = mean_ns.groupby(by=['BCLS'])['NS'].agg(['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_ns s ' + file_name
mean_ns.to_csv(file_name)
mean_ns = pd.read_csv(file_name, index_col=0)
display(mean_ns)

,mean
BCLS,
D,0.102273
E,0.560606
L,0.772727
N,0.220566
O,0.159091
Q,0.772727
R,0.089572
T,0.654545
X,0.454545


# Динамический профиль 

Исполнитель: Петриков А. Е.

In [7]:
start_date = datetime.strptime('2019-01-01', '%Y-%m-%d')
flight = 1126
filtered_data = hh[(hh['FLT_NUM'] == flight) & (hh['DD'] == start_date)]
days_ago = int(max(filtered_data['DTD'].unique()) + 1)
print("Дней в динамическом профиле", days_ago) # дней между началом продаж и днём вылета
end_date = (start_date - timedelta(days=days_ago)).strftime('%Y-%m-%d')
print("Дата начала продаж:", end_date) # дата начала продаж
filtered_data = hh[(hh['DD'] >= end_date) & (hh['DD'] <= start_date) & (hh['FLT_NUM'] == flight)]
dates_in_dyn_profile = filtered_data['DD'].unique()
print("Количество уникальных дат:", len(dates_in_dyn_profile))
# далее идёт выбор из этих дат

Дней в динамическом профиле 218
Дата начала продаж: 2018-05-28
Количество уникальных дат: 219


In [10]:
%%time
date = datetime.strptime('2018-12-07', '%Y-%m-%d') # input()
flttime = pd.Timedelta('21:20:00') # input()
print(date.date(), str(flttime).replace('0 days ', ''), date.strftime("%A"))
one_day = timedelta(days=1)
date_minus_1_week = pd.to_datetime(date) - pd.DateOffset(weeks=1)
date_minus_2_weeks = pd.to_datetime(date) - pd.DateOffset(weeks=2)
print(f'Выбранная дата: {date.date()}')
print(f'Дата -1 неделя: {date_minus_1_week.date()}')
print(f'Дата -2 недели: {date_minus_2_weeks.date()}')
# 1 неделя назад +- 1 день
date_minus_1_week_gap = [date_minus_1_week,
                         date_minus_1_week + one_day, date_minus_1_week - one_day]
# 2 недели назад +- 1 день
date_minus_2_weeks_gap = [date_minus_2_weeks,
                          date_minus_2_weeks + one_day, date_minus_2_weeks - one_day]
print(date_minus_1_week.date(), date_minus_2_weeks.date(), sep='\n')
# фильтрация по дням
filtered_hh = hh[(hh['DD'].isin(date_minus_1_week_gap)) | (hh['DD'].isin(date_minus_2_weeks_gap))]
print(f"Отфильтрованных данных: {filtered_hh.shape[0]:_}")
merged = pd.merge(bb, filtered_hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'], how='inner')
merged = merged.dropna(how='any', axis=0)
print(f"Объединённых данных: {merged.shape[0]:_}")
two_hours = pd.Timedelta('2:00:00')
# 1 неделя назад в заданное время +- 2 часа
date_minus_1_week_gap = [date_minus_1_week + flttime + two_hours,
                         date_minus_1_week + flttime - two_hours]
# 2 недели назад в заданное время +- 2 часа
date_minus_2_weeks_gap = [date_minus_2_weeks + flttime + two_hours,
                          date_minus_2_weeks + flttime - two_hours]
# фильтрация по времени +- 2 часа
merged = merged[(date_minus_1_week_gap[0] >= merged['DEPARTURE']) &
                (date_minus_1_week_gap[1] <= merged['DEPARTURE']) |
                (date_minus_2_weeks_gap[0] >= merged['DEPARTURE']) &
                (date_minus_2_weeks_gap[1] <= merged['DEPARTURE'])]
merged = merged.drop(columns=['DD', 'FLT_NUM'], axis=1)
merged = merged.dropna(how='any')
print(f"Отфильрованных по времени данных: {merged.shape[0]:_}")

2018-12-07 21:20:00 Friday
Выбранная дата: 2018-12-07
Дата -1 неделя: 2018-11-30
Дата -2 недели: 2018-11-23
2018-11-30
2018-11-23
Отфильтрованных данных: 567_703
Объединённых данных: 19_964_440
Отфильрованных по времени данных: 1_586_310
Wall time: 12.5 s


## Уникальные рейсы (рейс/дата/время) в сезонном профиле:

In [11]:
unique_flights = (merged['FLTNUM'].astype(str) + ' ' +
                 (merged['FLTDAT'] + merged['FLTTIME']).astype(str)).unique()
print(f'{len(unique_flights)} уникальных рейсов: {unique_flights}')

4 уникальных рейсов: ['1128 2018-11-30 21:20:00' '1130 2018-11-23 23:00:00'
 '1130 2018-11-30 23:00:00' '1128 2018-11-23 21:20:00']


## Средние цены тарифов для классов в сезонном профиле:

In [12]:
bcls_mean = merged[['BCLS', 'TARIF_RUB']].copy()
bcls_mean = bcls_mean.groupby(by=['BCLS'])['TARIF_RUB'].mean().reset_index()
bcls_mean

,BCLS,TARIF_RUB
0,C,34464.0
1,D,29504.0
2,E,3062.0
3,M,0.0
4,N,1905.0
5,O,0.0
6,Q,5732.0
7,R,751.0
8,T,4652.0


## Среднее количество броней (PASS_BK) в разрезе классов (BCLS) и дней до вылета (DTD):

In [13]:
mean_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                              aggfunc=['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_pass_bk d ' + file_name
mean_pass_bk.to_csv(file_name)
mean_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
mean_pass_bk = mean_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(mean_pass_bk)

std_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                             aggfunc=['std'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'std_pass_bk d ' + file_name
std_pass_bk.to_csv(file_name)
std_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
std_pass_bk = std_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(std_pass_bk)

BCLS,C,D,E,M,N,O,Q,R,T
DTD,,,,,,,,,
-1,5.227273,5.227273,7.272727,7.272727,6.339246,7.272727,5.227273,5.902893,5.227273
0,5.227273,5.227273,7.272727,7.272727,6.339246,7.272727,5.227273,5.902893,5.227273
1,5.454545,5.454545,7.045455,7.045455,5.717849,7.045455,5.454545,4.979339,5.454545
2,5.454545,5.454545,6.454545,6.454545,5.253880,6.454545,5.454545,4.603306,5.454545
3,5.363636,5.363636,6.227273,6.227273,4.896896,6.227273,5.363636,4.134298,5.363636
...,...,...,...,...,...,...,...,...,...
213,0.181818,0.181818,0.136364,0.136364,0.124723,0.136364,0.181818,0.115702,0.181818
214,0.136364,0.136364,0.136364,0.136364,0.116962,0.136364,0.136364,0.105372,0.136364
215,0.136364,0.136364,0.136364,0.136364,0.116962,0.136364,0.136364,0.105372,0.136364


BCLS,C,D,E,M,N,O,Q,R,T
DTD,,,,,,,,,
-1,10.188602,10.129878,15.338675,15.492479,14.222040,15.403185,10.100894,14.022027,10.129878
0,10.129878,10.100894,15.327032,15.403185,14.220069,15.359113,10.086495,14.018403,10.100894
1,10.826648,10.795670,14.799040,14.872569,13.153956,14.830015,10.780281,12.086834,10.795670
2,10.852098,10.821047,14.735073,14.808284,12.761134,14.765914,10.805622,11.569526,10.821047
3,10.770524,10.739707,14.636512,14.709233,11.957480,14.667147,10.724397,10.131974,10.739707
...,...,...,...,...,...,...,...,...,...
213,0.492366,0.490957,0.457159,0.459430,0.447361,0.458116,0.490257,0.439687,0.490957
214,0.459430,0.458116,0.457159,0.459430,0.440732,0.458116,0.457463,0.430434,0.458116
215,0.459430,0.458116,0.457159,0.459430,0.440732,0.458116,0.457463,0.430434,0.458116


## Среднее количество улетевших пассажиров (PASS_DEP) в разрезе классов (BCLS) и дней до вылета (DTD) в момент времени -1:

In [14]:
mean_std_pass_dep = merged[merged['DTD'] == -1]
mean_std_pass_dep = mean_std_pass_dep.groupby(by=['BCLS'])['PASS_DEP'].agg(['mean', 'std'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_std_pass_dep d ' + file_name
mean_std_pass_dep.to_csv(file_name)
mean_std_pass_dep = pd.read_csv(file_name, index_col=0)
display(mean_std_pass_dep)

,mean,std
BCLS,,
C,5.227273,10.188602
D,5.227273,10.129878
E,7.272727,15.338675
M,7.272727,15.492479
N,6.223947,13.837972
O,7.272727,15.403185
Q,5.227273,10.100894
R,5.704545,13.382508
T,5.227273,10.129878


## Среднее количество неявившихся пассажиров (NS) в разрезе классов (BCLS) и дней до вылета (DTD):

In [15]:
mean_ns = merged[merged['DTD'] == -1]
mean_ns = mean_ns.groupby(by=['BCLS'])['NS'].agg(['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_ns d ' + file_name
mean_ns.to_csv(file_name)
mean_ns = pd.read_csv(file_name, index_col=0)
display(mean_ns)

,mean
BCLS,
C,0.272727
D,0.272727
E,0.045455
M,0.045455
N,0.093126
O,0.045455
Q,0.272727
R,0.121901
T,0.272727
